In [ ]:
from fsspec.implementations.http import HTTPFileSystem
import holoviews as hv
import hvplot.xarray  # noqa: F401

from emit_tools import emit_xarray
from utils import get_earthdata_token

## Band animations

This notebook will create an interactive visualisation explorer or animation that
steps through the bands.

Since the hyperspectral sensor splits the spectrum into 285 bands, there are a lot
of combinations that could be used if we stick to RGB or single-band visualisation
as demonstrated in the first notebook. This notebook will allow us to quickly
explore the scene across the spectra. 

In [ ]:
# See README.md for instructions on how to get an Earthdata token
token = get_earthdata_token()

In [ ]:
%%time
# Loading data can take around 4 minutes on a 100Mbps connection

# kernel = "EMIT_L2A_RFL_001_20230123T004529_2302216_003" # Hobart
kernel = "EMIT_L2A_RFL_001_20230316T045211_2307503_006" # Canberra

s3_url = "s3://lp-prod-protected/EMITL2ARFL.001/" + kernel + "/" + kernel + ".nc"
s3_url = s3_url.replace("s3://", "https://data.lpdaac.earthdatacloud.nasa.gov/")

fs = HTTPFileSystem(headers={
    "Authorization": f"bearer {token}"
})
ds = emit_xarray(fs.open(s3_url))
ds

In [ ]:
# Render an image for each band and build an interactive explorer
# This takes a few minutes to render if you ask for many images

# All wavelengths... generally doesn't work because there's too many!
# wavelengths = ds.wavelengths.values
        
# A subset of wavelengths. Change the third value to change the step size.
wavelengths = ds.wavelengths.values[range(0, 285, 20)]

animation = hv.HoloMap(
    {
        band: hv.Image(ds.sel(bands=band).reflectance).opts(
            cmap="viridis", colorbar=True, width=600, height=400, xaxis='bare', yaxis='bare', invert_axes=True
        )
        for band in wavelengths
    },
    kdims="band",
)
# Uncomment the below to view the interactive explorer, though it crashes with lots of images...
# animation

In [ ]:
# Inline animated output
hv.output(animation, holomap="gif", fps=4)

In [ ]:
# Save animation as a file.
hv.save(animation, 'spectral_animation.gif', fps=4, toolbar=False)